In [1]:
import cv2
import torch
import numpy as np
import json
import os
import mediapipe as mp
from pathlib import Path
from models.common import DetectMultiBackend
from utils.general import non_max_suppression

In [2]:
# Khởi tạo MediaPipe
mp_pose = mp.solutions.pose.Pose(min_detection_confidence=0.6, min_tracking_confidence=0.6)
mp_hands = mp.solutions.hands.Hands(static_image_mode=False, max_num_hands=2)


In [3]:
# Load YOLOv3 model
device = torch.device("cpu")
yolo_model = DetectMultiBackend("weights/yolov3-tiny.pt")
yolo_model.to(device).eval()


Fusing layers... 
yolov3-tiny summary: 48 layers, 8849182 parameters, 0 gradients, 13.2 GFLOPs


DetectMultiBackend(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv(
        (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv(
   

In [4]:
# Tạo thư mục lưu kết quả
output_dir = Path("runs/detect/frames")
output_dir.mkdir(parents=True, exist_ok=True)
video_output_path = "runs/detect/output_video.mp4"
json_output_path = "runs/detect/sign_space.json"
json_output = {}

In [5]:
# Kích thước YOLO yêu cầu (mặc định 640x640)
YOLO_IMG_SIZE = 640

In [18]:
def resize_frame(frame):
    """Resize frame về kích thước YOLO yêu cầu nhưng vẫn lưu tỷ lệ ảnh gốc."""
    original_size = frame.shape[:2]  # (height, width)
    resized_frame = cv2.resize(frame, (YOLO_IMG_SIZE, YOLO_IMG_SIZE))
    return resized_frame, original_size

In [6]:
def detect_person(model, frame):
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = torch.from_numpy(img).to(device).permute(2, 0, 1).float().div(255.0).unsqueeze(0)
    
    with torch.no_grad():
        pred = model(img)
    
    detections = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, classes=[0])
    
    print(f"[DEBUG] Detections: {detections}")  # Kiểm tra kết quả YOLO


    if detections[0] is not None:
        return detections[0].cpu().numpy()
    return []

In [ ]:
def get_pose_and_hand_landmarks(frame):
    """Lấy tọa độ pose và bàn tay từ MediaPipe"""
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Xử lý pose landmarks
    results_pose = mp_pose.process(frame_rgb)
    pose_landmarks = []
    if results_pose.pose_landmarks:
        pose_landmarks = [(lm.x * frame.shape[1], lm.y * frame.shape[0]) for lm in results_pose.pose_landmarks.landmark]
    
    # Xử lý hand landmarks
    results_hands = mp_hands.
    
    
    (frame_rgb)

    left_hand_landmarks = []
    right_hand_landmarks = []
    
    if results_hands.multi_hand_landmarks and results_hands.multi_handedness:
        for hand_landmarks, handedness in zip(results_hands.multi_hand_landmarks, results_hands.multi_handedness):
            label = handedness.classification[0].label  # "Left" hoặc "Right"
            landmarks = [(lm.x * frame.shape[1], lm.y * frame.shape[0]) for lm in hand_landmarks.landmark]
            
            # Quan trọng: Đây là góc nhìn của camera, nên "Left" thực sự là tay phải của người
            # và "Right" thực sự là tay trái của người
            if label == "Left":  # Tay phải của người (từ góc nhìn camera)
                right_hand_landmarks = landmarks
            elif label == "Right":  # Tay trái của người (từ góc nhìn camera)
                left_hand_landmarks = landmarks

    return pose_landmarks, left_hand_landmarks, right_hand_landmarks

In [8]:
def calculate_head_unit(pose_landmarks, person_box, original_size):
    height, width = original_size
    x1, y1, x2, y2 = person_box[:4]
    
    if pose_landmarks and len(pose_landmarks) >= 6:
        left_eye, right_eye = pose_landmarks[2], pose_landmarks[5]
        head_unit = np.sqrt((left_eye[0] - right_eye[0])**2 + (left_eye[1] - right_eye[1])**2)
    else:
        head_unit = ((x2 - x1) / width + (y2 - y1) / height) / 2 * height / 6
    
    # print(f"[INFO] Head Unit: {head_unit}")  # Debug head_unit
    return head_unit

In [9]:
def calculate_hand_bbox_from_mediapipe(hand_landmarks, img_shape, padding=15):
    """Tính toán bounding box cho bàn tay dựa trên toàn bộ landmark, không chỉ cổ tay"""
    if not hand_landmarks:
        return None

    height, width = img_shape[:2]
    
    x_coords = [int(landmark[0]) for landmark in hand_landmarks]
    y_coords = [int(landmark[1]) for landmark in hand_landmarks]

    x_min, y_min = max(0, min(x_coords) - padding), max(0, min(y_coords) - padding)
    x_max, y_max = min(width, max(x_coords) + padding), min(height, max(y_coords) + padding)

    return [x_min, y_min, x_max, y_max]

In [13]:
def calculate_anchor_boxes(sign_space, hand_boxes, original_size, resized_size, confidence=0.5):
    """
    Tính toán các hộp neo chuẩn hóa cho không gian ký hiệu và bàn tay
    
    Parameters:
    sign_space (list): Bounding box của không gian ký hiệu [x1, y1, x2, y2]
    hand_boxes (dict): Dictionary chứa bounding box của tay trái và phải
    original_size (tuple): Kích thước gốc của video (height, width)
    resized_size (tuple): Kích thước của frame sau khi resize (height, width)
    confidence (float): Độ tin cậy của dự đoán (mặc định 0.5)
    
    Returns:
    list: Danh sách các hộp neo chuẩn hóa
    """
    anchor_boxes = []
    
    # Thêm hộp neo cho không gian ký hiệu nếu có
    if sign_space:
        normalized_sign_space = normalize_bbox(sign_space, original_size, resized_size)
        anchor_boxes.append({
            "type": "sign_space",
            "bbox": normalized_sign_space,
            "confidence": confidence
        })
    
    # Thêm hộp neo cho tay trái nếu có
    if hand_boxes.get("left"):
        normalized_left_hand = normalize_bbox(hand_boxes["left"], original_size, resized_size)
        anchor_boxes.append({
            "type": "left_hand",
            "bbox": normalized_left_hand,
            "confidence": confidence
        })
    
    # Thêm hộp neo cho tay phải nếu có
    if hand_boxes.get("right"):
        normalized_right_hand = normalize_bbox(hand_boxes["right"], original_size, resized_size)
        anchor_boxes.append({
            "type": "right_hand",
            "bbox": normalized_right_hand,
            "confidence": confidence
        })
    
    return anchor_boxes

In [ ]:
def normalize_bbox(bbox, original_size, resized_size, confidence=0.5):
    """
    Chuẩn hóa bounding box từ frame đã resize về tọa độ chuẩn hóa [x_center, y_center, width, height]
    dựa trên kích thước gốc của video
    
    Parameters:
    bbox (list): Bounding box ở định dạng [x1, y1, x2, y2] trên frame đã resize
    original_size (tuple): Kích thước gốc của video (height, width)
    resized_size (tuple): Kích thước của frame sau khi resize (height, width)
    
    Returns:
    list: Bounding box chuẩn hóa ở định dạng [x_center, y_center, width, height]
    """
    x1, y1, x2, y2 = bbox
    original_height, original_width = original_size
    resized_height, resized_width = resized_size
    
    # Chuyển đổi tọa độ từ frame đã resize về tỷ lệ tương ứng trong frame gốc
    x1_ratio = x1 / resized_width
    y1_ratio = y1 / resized_height
    x2_ratio = x2 / resized_width
    y2_ratio = y2 / resized_height
    
    # Tính toán tọa độ trung tâm và kích thước (đã được chuẩn hóa)
    x_center_norm = (x1_ratio + x2_ratio) / 2.0
    y_center_norm = (y1_ratio + y2_ratio) / 2.0
    width_norm = x2_ratio - x1_ratio
    height_norm = y2_ratio - y1_ratio
    
    return [x_center_norm, y_center_norm, width_norm, height_norm,confidence]

In [12]:
def calculate_normalized_sign_space(frame, pose_landmarks, head_unit):
    if not pose_landmarks or head_unit is None:
        return None
    nose, left_eye, right_eye = pose_landmarks[0], pose_landmarks[2], pose_landmarks[5]
    # Kích thước box (theo tài liệu là 7 - 8)
    width = 7 * head_unit
    height = 8 * head_unit 
    
    # Dùng trung bình giữa mũi và mắt để ổn định center_x
    center_x = int(nose[0])  # Tâm ngang chính xác theo mũi
    # center_y = int((nose[1] + left_eye[1]) / 2)  # Trung bình giữa mũi và mắt trái
    center_y = int(left_eye[1])  # Định vị theo mắt trái

    
    x1 = int(center_x - width / 2)
    # Lùi lên 1.5 đơn vị đầu (Theo tài liệu 0.5)
    y1 = int(center_y - 1.5 * head_unit)  # Điều chỉnh vị trí y1
    x2 = int(center_x + width / 2)
    y2 = min(frame.shape[0] - 1, int(center_y + 7.5 * head_unit))  # Cạnh dưới cách mắt trái 7.5 head_unit
    
    h, w, _ = frame.shape
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(w - 1, x2), min(h - 1, y2)
    print(f"[INFO] Sign Space Box: ({x1}, {y1}), ({x2}, {y2})")  # Debug bbox
    return [x1, y1, x2, y2]

In [11]:
def draw_bbox_on_image(img, bbox, color=(0, 255, 0), thickness=2, label="Hand Anchor"):
    """Vẽ bounding box lên hình ảnh"""
    if bbox is None:
        return img
    x1, y1, x2, y2 = bbox
    cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)
    cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return img


In [14]:
def process_frame(frame, frame_index):
    resized_frame, original_size = resize_frame(frame)
    detections = detect_person(yolo_model, resized_frame)
    
    if len(detections) == 0:
        print("[DEBUG] Không phát hiện người trong frame!")
        return frame, None

    # Lấy pose + landmark tay từ MediaPipe
    pose_landmarks, left_hand_landmarks, right_hand_landmarks = get_pose_and_hand_landmarks(resized_frame)
    
    head_unit = calculate_head_unit(pose_landmarks, detections[0], original_size)
    sign_space = calculate_normalized_sign_space(resized_frame, pose_landmarks, head_unit)

    # Tính toán bounding box tay
    left_hand_box = calculate_hand_bbox_from_mediapipe(left_hand_landmarks, resized_frame.shape) if left_hand_landmarks else None
    right_hand_box = calculate_hand_bbox_from_mediapipe(right_hand_landmarks, resized_frame.shape) if right_hand_landmarks else None

    # Vẽ hộp không gian ký hiệu
    if sign_space:
        x1, y1, x2, y2 = sign_space
        cv2.rectangle(resized_frame, (x1, y1), (x2, y2), (255, 255, 0), 2)  # Màu xanh dương nhạt

    # Vẽ hộp tay
    if left_hand_box:
        draw_bbox_on_image(resized_frame, left_hand_box, color=(255, 0, 0), label="Left Hand")
    if right_hand_box:
        draw_bbox_on_image(resized_frame, right_hand_box, color=(0, 0, 255), label="Right Hand")

    print(f"[DEBUG] Sign Space: {sign_space}")  # Kiểm tra tọa độ sign space


    return resized_frame, sign_space

In [15]:
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Không thể mở video: {video_path}")
        return
    
    # Lấy thông tin video để tạo video output
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Tạo video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(video_output_path, fourcc, fps, (YOLO_IMG_SIZE, YOLO_IMG_SIZE))
    
    frame_index = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Xử lý frame
        processed_frame, sign_space = process_frame(frame, frame_index)
        
        # Ghi frame đã xử lý vào video output
        out.write(processed_frame)
        
        # Hiển thị frame
        cv2.imshow('Sign Space & Hand Detection', processed_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # Lưu thông tin sign_space vào json_output nếu có
        if sign_space:
            json_output[str(frame_index)] = sign_space
        
        frame_index += 1

    # Giải phóng tài nguyên
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    # Lưu json_output ra file
    with open(json_output_path, 'w') as f:
        json.dump(json_output, f)


In [19]:
if __name__ == "__main__":
    process_video("E:/University/HK2_Nam3/PBL/clone_yolo/yolov3/videos_demo/demo_2.mp4")

[DEBUG] Detections: [tensor([[1.50357e+02, 1.07487e+02, 5.15583e+02, 5.68277e+02, 2.79987e-01, 0.00000e+00]])]
[INFO] Sign Space Box: (137, 82), (516, 571)
[DEBUG] Sign Space: [137, 82, 516, 571]
[DEBUG] Detections: [tensor([[1.53739e+02, 1.08097e+02, 5.10761e+02, 5.72974e+02, 2.61660e-01, 0.00000e+00]])]
[INFO] Sign Space Box: (136, 81), (517, 570)
[DEBUG] Sign Space: [136, 81, 517, 570]
[DEBUG] Detections: [tensor([[1.52560e+02, 1.08857e+02, 5.09896e+02, 5.72519e+02, 2.57786e-01, 0.00000e+00]])]
[INFO] Sign Space Box: (136, 81), (517, 571)
[DEBUG] Sign Space: [136, 81, 517, 571]
[DEBUG] Detections: [tensor([[1.52601e+02, 1.08680e+02, 5.09789e+02, 5.72566e+02, 2.56017e-01, 0.00000e+00]])]
[INFO] Sign Space Box: (137, 81), (518, 571)
[DEBUG] Sign Space: [137, 81, 518, 571]
[DEBUG] Detections: [tensor([[1.53298e+02, 1.07742e+02, 5.11598e+02, 5.69587e+02, 2.62489e-01, 0.00000e+00]])]
[INFO] Sign Space Box: (137, 82), (518, 571)
[DEBUG] Sign Space: [137, 82, 518, 571]
[DEBUG] Detections: 